# 외부 데이터 수집과 정제

In [1]:
import requests
import lxml.html

In [2]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('.view_box a'):
    url = a.get('href')
    print(url)

/store/books/look.php?p_code=B4416755821
/store/books/look.php?p_code=B4416755821
/store/books/look.php?p_code=B8753305821
/store/books/look.php?p_code=B8753305821
/store/books/look.php?p_code=B9612872259
/store/books/look.php?p_code=B9612872259
/store/books/look.php?p_code=B6367128158
javascript:;
/store/books/look.php?p_code=B6367128158
/store/books/look.php?p_code=B2998156507
/store/books/look.php?p_code=B2998156507
/store/books/look.php?p_code=B8839851696
javascript:;
/store/books/look.php?p_code=B8839851696
/store/books/look.php?p_code=B5373757319
javascript:;
/store/books/look.php?p_code=B5373757319
/store/books/look.php?p_code=B2369016915
javascript:;
/store/books/look.php?p_code=B2369016915
/store/books/look.php?p_code=B9992267828
/store/books/look.php?p_code=B9992267828
/store/books/look.php?p_code=B9564731917
/store/books/look.php?p_code=B9564731917
/store/books/look.php?p_code=B4153780113
/store/books/look.php?p_code=B4153780113
/store/books/look.php?p_code=B7179675656
javas

In [3]:
import requests
import lxml.html
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

In [4]:
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

https://www.hanbit.co.kr/store/books/look.php?p_code=B4416755821
https://www.hanbit.co.kr/store/books/look.php?p_code=B8753305821
https://www.hanbit.co.kr/store/books/look.php?p_code=B9612872259
https://www.hanbit.co.kr/store/books/look.php?p_code=B6367128158
https://www.hanbit.co.kr/store/books/look.php?p_code=B2998156507
https://www.hanbit.co.kr/store/books/look.php?p_code=B8839851696
https://www.hanbit.co.kr/store/books/look.php?p_code=B5373757319
https://www.hanbit.co.kr/store/books/look.php?p_code=B2369016915
https://www.hanbit.co.kr/store/books/look.php?p_code=B9992267828
https://www.hanbit.co.kr/store/books/look.php?p_code=B9564731917
https://www.hanbit.co.kr/store/books/look.php?p_code=B4153780113
https://www.hanbit.co.kr/store/books/look.php?p_code=B7179675656
https://www.hanbit.co.kr/store/books/look.php?p_code=B7687052472
https://www.hanbit.co.kr/store/books/look.php?p_code=B8671421572
https://www.hanbit.co.kr/store/books/look.php?p_code=B8341772179
https://www.hanbit.co.kr/

In [5]:
import requests
import lxml.html
import re
import time

In [6]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [7]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [8]:
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B4416755821
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8753305821
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B9612872259
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B6367128158
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B2998156507
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8839851696
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B5373757319
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

In [9]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [10]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [11]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1)
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        #break
    
    

In [ ]:
if __name__ == '__main__':
    main()

In [13]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [14]:
REG_DATE = '20200819'

In [15]:
response = requests.get('https://news.daum.net/breakingnews/digital?regDate={}'.format(REG_DATE))                   
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200819230943259 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230922257 트럼프 "오라클은 대단한 회사"..틱톡 인수 지지
https://v.daum.net/v/20200819230858252 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230757249 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230752248 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230732247 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230639242 한국심리학회 연차학술대회 20~22일 열려
https://v.daum.net/v/20200819230626241 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230623240 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230557231 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230516226 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230512224 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230448221 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819225027087 글로벌창업사

In [16]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [17]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [18]:
page = 1
max_page = 0
REG_DATE = '20200819'

In [ ]:
response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                        .format(page, REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    article = get_detail(url)
    print(f'URL : {url}')
    print(f'TITLE : {a.text}')
    print(f'ARTICLE : {article}')
    print('-' * 100)

In [20]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [21]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [22]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [23]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST   


In [24]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [25]:
page = 58
max_page = 0
REG_DATE = '20200819'

In [26]:
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [article]})
        df_list.append(df)   
        
    if df_list:   
        df_10 = pd.concat(df_list)
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기    
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num       
        except:
            pass

    # 마지막 페이지 여부 확인     
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1
        
    time.sleep(1)      

15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
13 건 저장완료..


In [27]:
print(db_select())

                                       URL  \
0   https://v.daum.net/v/20200819084702255   
1   https://v.daum.net/v/20200819084617234   
2   https://v.daum.net/v/20200819084529215   
3   https://v.daum.net/v/20200819084517208   
4   https://v.daum.net/v/20200819084251134   
5   https://v.daum.net/v/20200819083702982   
6   https://v.daum.net/v/20200819083528956   
7   https://v.daum.net/v/20200819083303903   
8   https://v.daum.net/v/20200819083135880   
9   https://v.daum.net/v/20200819082825812   
10  https://v.daum.net/v/20200819082814806   
11  https://v.daum.net/v/20200819082703784   
12  https://v.daum.net/v/20200819082610764   
13  https://v.daum.net/v/20200819082557759   
14  https://v.daum.net/v/20200819082503742   
15  https://v.daum.net/v/20200819082501741   
16  https://v.daum.net/v/20200819082324702   
17  https://v.daum.net/v/20200819081608550   
18  https://v.daum.net/v/20200819080415266   
19  https://v.daum.net/v/20200819080203202   
20  https://v.daum.net/v/202008190